In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import requests
from bs4 import BeautifulSoup




In [3]:
URL = ["https://www.imdb.com/search/title/?genres=action&explore=title_type,genres&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=e0da8c98-35e8-4ebd-8e86-e7d39c92730c&pf_rd_r=NCWR107JR3D070SM7GBG&pf_rd_s=center-2&pf_rd_t=15051&pf_rd_i=genre&ref_=ft_gnr_pr2_i_2",
                "https://www.imdb.com/search/title/?genres=action&start=51&explore=title_type,genres&ref_=adv_nxt"]
  


In [40]:
# Creating the lists that we want to 

description = []
duration=[]
release_date=[]
movie_director_stars=[]
movie_years=[]
director_sep = []
stars_sep = []


#movie_name_info100=extract_data.find

movie_name_100=[]

for url in range(0,2):
    req = requests.get(URL[url])
    soup = BeautifulSoup(req.text, 'html.parser')
    for container in soup.find_all("h3" ,class_="lister-item-header"):
        movie_name_100.append(container.text.replace('\n', '').strip(')').split('(')[0]) # Extracting tthe top 100 movies 
        release_date.append(container.text.replace('\n', '').strip(')').split('(')[1]) # 
    for release in soup.find_all('span', class_="runtime"):
        duration.append(release.text) 

    for dist in soup.find_all('p', class_="text-muted"):
        description.append(dist.text.replace('\n', '').split()[1][:])

    for director in soup.find_all('p', class_ =''):
        director_sep.append(director.text.replace("\n", "").split('|')[0].split(':')[1].split(',')[0])
        stars_sep.append(director.text.replace("\n",'').split('Stars:')[1])

    for years in soup.find_all('span', class_ ='lister-item-year'):
        movie_years.append(years.text)
    
        
# print(director_sep)
print(stars_sep)




['Robert Pattinson, Zoë Kravitz, Jeffrey Wright, Colin Farrell', 'Ryan Reynolds, Walker Scobell, Mark Ruffalo, Jennifer Garner', 'Alexander Dreymon, Eliza Butterworth, Arnas Fedaravicius, Mark Rowley', 'Rupert Friend, Hayden Christensen, Ewan McGregor, Joel Edgerton', 'Sam Corlett, Leo Suter, Jóhannes Haukur Jóhannesson, Lujza Richter', 'Tom Holland, Zendaya, Benedict Cumberbatch, Jacob Batalon', 'Rhys Darby, Joel Fry, Samson Kayo, Nathan Foad', 'Ralph Fiennes, Gemma Arterton, Rhys Ifans, Harris Dickinson', 'Yûki Kaji, Marina Inoue, Yui Ishikawa, Josh Grelle', 'Zazie Beetz, Brad Pitt, Aaron Taylor-Johnson, Sandra Bullock', 'Alan Ritchson, Malcolm Goodwin, Willa Fitzgerald, Hugh Thompson', 'Christian Bale, Heath Ledger, Aaron Eckhart, Michael Caine', 'Patrick Stewart, Alison Pill, Michelle Hurd, Isa Briones', 'Tom Holland, Mark Wahlberg, Antonio Banderas, Sophia Ali', 'Katheryn Winnick, Gustaf Skarsgård, Alexander Ludwig, Georgia Hirst', 'Emilia Clarke, Peter Dinklage, Kit Harington, Le